# 커스텀 Document Loader 작성 방법

# 개요

LLM 기반 애플리케이션은 PDF 같은 파일 또는 데이터베이스로부터 데이터를 추출하고 LLM이 활용할 수 있는 형태로 변환해야 한다. `랭체인(LangChain)`에서는 일반적으로 추출된 `텍스트(page_content)`를 `메타데이터(metadata)`와 함께 캡슐화하는 `Document` 객체(작가의 이름이나 게시 날짜 등 문서에 대한 세부 정보가 포함된 사전)를 생성한다.

`Document` 객체는 LLM에 입력되는 프롬프트로 변환된다. 그리고 LLM은 이 `Document` 내의 정보를 사용하여 원하는 응답(예: 문서 요약)을 생성한다. `Document` 객체는 즉시 사용하거나 향후 검색을 위해 `벡터 저장소(vectorstore)`에 저장된다.

문서 로딩의 주요 추상화는 다음과 같다.
- Document : `text`와 `metadata`를 가진다.
- BaseLoader : 원본 데이터를 `Document` 리스트로 변환시킨다.
- Blob : 파일 또는 메모리에 존재하는 이진 데이터를 나타낸다.
- BaseBlobParser : `Blob`을 파싱하여 문서 객체를 생성한다.

이 가이드에서는 `사용자 지정 문서 로딩(custom document loading)`과 `파일 구문 분석 로직(file parsing logic)`을 작성하는 방법을 설명한다.

1. `BaseLoader` 인터페이스를 구현한 `표준 문서 로더(standard document loader)`를 만든다.
2. `BaseBlobParser`를 사용하여 파서를 생성하고 `Blob` 및 `BlobLoaders`와 함께 사용한다. 이 기능은 주로 파일 작업 시 유용하다.


# 표준 문서 로더
`문서 로더(document loader)`는 문서 로딩을 위한 표준 인터페이스를 제공하는 `BaseLoader` 인터페이스를 구현한다.

## 인터페이스
- lazy_load : `지연 로딩(lazy loading)` 방식으로 문서를 하나씩 로드한다. 이 메소드는 프로덕션 코드에서 사용한다.
- alazy_load : `lazy_load` 메소드의 비동기화 버전이다. 
- load : `즉시 로딩(eager loading)` 방식으로 모든 문서를 메모리에 로드한다. 이 메소드는 개발 코드에서 사용한다.
- aload : `load` 메소드의 비동기화 버전이다.

`load`와 `aload` 메소드는 개발 작업만을 위한 편의성 방법으로, 내부적으로 `list(self.lazy_load())`를 호출한다.

`alazy_load` 메소드는 내부적으로  `lazy_load` 메소드를 호출한다. 따라서 비동기를 사용하는 경우 `lazy_load` 메소드 구현을 재정의하고 `alazy_load` 메소드를 사용하는 것이 좋다.

문서 로더를 구현할 때 `lazy_load` 또는 `lazy_load` 메서드를 통해 매개 변수를 제공하지 않는다.

따라서 모든 구성은 `초기화(init)` 메소드를 통해 전달되고 이는 문서 로더가 인스턴스화되면 문서를 로드하는 데 필요한 모든 정보를 가질 수 있도록 LangChain이 설계한 것이다.

## 참고
- https://en.wikipedia.org/wiki/Lazy_loading
- https://python.langchain.com/api_reference/core/document_loaders/langchain_core.document_loaders.base.BaseLoader.html
- https://python.langchain.com/api_reference/_modules/langchain_core/document_loaders/base.html#BaseLoader

## 구현

파일을 로드하고 파일의 각 줄에서 문서를 생성하는 표준 문서 로더의 예를 만들어보자.

In [24]:
pip install -qU langchain-core aiofiles ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [2]:
from typing import AsyncIterator, Iterator

from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document

class CustomDocumentLoader(BaseLoader):
    """An example document loader that reads a file line by line."""

    def __init__(self, file_path: str) -> None:
        """Initialize the loader with a file path.

        Args:
            file_path: The path to the file to load.
        """
        self.file_path = file_path
    
    # Does not take any arguments
    def lazy_load(self) -> Iterator[Document]:
        """A lazy loader that reads a file line by line.

        When you're implementing lazy load methods, 
        you should use a generator to yield documents one by one
        """
        with open(self.file_path, encoding="utf-8") as f:
            line_number = 0
            for line in f:
                yield Document(
                    page_content=line,
                    metadata={
                        "line_number": line_number, 
                        "source": self.file_path
                    }
                )
                line_number += 1
    
    # alazy_load is OPTIONAL.
    # If you leave out the implementation, 
    # a default implementation which delegates to lazy_load will be used!
    async def alazy_load(
        self,
    ) -> AsyncIterator[Document]:   # Does not take any arguments
        """An async lazy loader that reads a file line by line."""
        
        # Requires aiofiles
        # Install with `pip install aiofiles`
        import aiofiles

        async with aiofiles.open(self.file_path, encoding="utf-8") as f:
            line_number = 0
            async for line in f:
                yield Document(
                    page_content=line,
                    metadata={
                        "line_number": line_number,
                        "source": self.file_path,
                    }
                )
                line_number += 1
        


## 테스트

In [5]:
with open("./meow.txt", "w", encoding="utf-8") as f:
    quality_content = "meow meow🐱 \n meow meow🐱 \n meow😻😻"
    f.write(quality_content)

loader = CustomDocumentLoader("./sample_data/meow.txt")

In [6]:
## Test out the lazy load interface
for doc in loader.lazy_load():
    print()
    print(type(doc))
    print(doc)


<class 'langchain_core.documents.base.Document'>
page_content='meow meow🐱 
' metadata={'line_number': 0, 'source': './sample_data/meow.txt'}

<class 'langchain_core.documents.base.Document'>
page_content=' meow meow🐱 
' metadata={'line_number': 1, 'source': './sample_data/meow.txt'}

<class 'langchain_core.documents.base.Document'>
page_content=' meow😻😻' metadata={'line_number': 2, 'source': './sample_data/meow.txt'}


In [7]:
## Test out the async implementation
async for doc in loader.alazy_load():
    print()
    print(type(doc))
    print(doc)


<class 'langchain_core.documents.base.Document'>
page_content='meow meow🐱 
' metadata={'line_number': 0, 'source': './sample_data/meow.txt'}

<class 'langchain_core.documents.base.Document'>
page_content=' meow meow🐱 
' metadata={'line_number': 1, 'source': './sample_data/meow.txt'}

<class 'langchain_core.documents.base.Document'>
page_content=' meow😻😻' metadata={'line_number': 2, 'source': './sample_data/meow.txt'}


load 메소드는 주피터 노트북과 같은 대화형 환경에서 유용하지만 엔터프라이즈 데이터의 경우 모든 데이터가 메모리에 들어갈 수 있다고 가정하기 때문에 프로덕션 환경에서는 사용하지 말아야 한다.

In [8]:
loader.load()

[Document(metadata={'line_number': 0, 'source': './sample_data/meow.txt'}, page_content='meow meow🐱 \n'),
 Document(metadata={'line_number': 1, 'source': './sample_data/meow.txt'}, page_content=' meow meow🐱 \n'),
 Document(metadata={'line_number': 2, 'source': './sample_data/meow.txt'}, page_content=' meow😻😻')]

# 로딩과 구문 분석

많은 `문서 로더(document loader)`에는 파일 구문 분석이 포함된다. 이러한 로더의 차이점은 일반적으로 `파일이 로드되는 방식(load logic)`이 아니라 `파일이 구문 분석되는 방식(parsing logic)`에서 비롯된다. 예를 들어, `open`을 사용하여 PDF 또는 마크다운 파일의 이진 내용을 읽을 수 있지만, 이진 데이터를 텍스트로 변환하려면 다른 구문 분석 로직이 필요하다.

따라서 `구문 분석 로직(parsing logic)`과 `로딩 로직(loading logic)`을 분리하면 데이터가 어떻게 로딩되었는지에 관계없이 주어진 `구문 분석기(parser)`를 더 쉽게 재사용할 수 있다.

## 참고
- https://python.langchain.com/api_reference/_modules/langchain_community/document_loaders/pdf.html#PyPDFLoader

## BaseBlobParser

`BaseBlobParser`는 `blob`을 받아서 `Document` 객체 목록을 출력하는 인터페이스다. `blob`은 메모리 또는 파일에 저장된 데이터를 표현한다.

In [9]:
from langchain_core.document_loaders import BaseBlobParser, Blob

class MyParser(BaseBlobParser):
    """A simple parser that creates a document from each line."""

    def lazy_parse(self, blob: Blob) -> Iterator[Document]:
        """Parse a blob into a document line by line."""
        line_number = 0
        with blob.as_bytes_io() as f:
            for line in f:
                line_number += 1
                yield Document(
                    page_content=line,
                    metadata={
                        "line_number": line_number,
                        "source": blob.source,
                    }
                )

In [11]:
blob = Blob.from_path("./sample_data/meow.txt")
parser = MyParser()

In [12]:
list(parser.lazy_parse(blob))

[Document(metadata={'line_number': 1, 'source': './sample_data/meow.txt'}, page_content='meow meow🐱 \r\n'),
 Document(metadata={'line_number': 2, 'source': './sample_data/meow.txt'}, page_content=' meow meow🐱 \r\n'),
 Document(metadata={'line_number': 3, 'source': './sample_data/meow.txt'}, page_content=' meow😻😻')]

또한 `blob API`를 사용하면 파일에서 읽을 필요 없이 메모리에서 바로 콘텐츠를 로드할 수 있다.

In [13]:
blob = Blob(data=b"some data from memory\nmeow")
list(parser.lazy_parse(blob))

[Document(metadata={'line_number': 1, 'source': None}, page_content='some data from memory\n'),
 Document(metadata={'line_number': 2, 'source': None}, page_content='meow')]

## Blob

In [14]:
blob = Blob.from_path("./sample_data/meow.txt", metadata={"foo": "bar"})

In [15]:
blob.encoding

'utf-8'

In [16]:
blob.as_bytes()

b'meow meow\xf0\x9f\x90\xb1 \r\n meow meow\xf0\x9f\x90\xb1 \r\n meow\xf0\x9f\x98\xbb\xf0\x9f\x98\xbb'

In [18]:
blob.as_string()

'meow meow🐱 \n meow meow🐱 \n meow😻😻'

In [19]:
blob.as_bytes_io()

In [20]:
blob.metadata

{'foo': 'bar'}

In [21]:
blob.source

'./sample_data/meow.txt'

## Blob 로더

`구문 분석기(parser)`는 이진 데이터를 문서로 구문 분석하는 데 필요한 로직을 캡슐화하는 반면, `Blob 로더`는 주어진 저장 위치에서 `Blob`을 로드하는 데 필요한 로직을 캡슐화한다.

현재 LangChain은 0.3 버전 기준으로 `FileSystemBlobLoader`만 지원한다. `FileSystemBlobLoader`를 사용하여 `Blob`을 로드한 다음 구문 분석기를 사용하여 `Blob`을 구문 분석할 수 있습니다.

## 참고
- https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.blob_loaders.file_system.FileSystemBlobLoader.html

In [28]:
from langchain_community.document_loaders.blob_loaders import FileSystemBlobLoader

blob_loader = FileSystemBlobLoader(path="./sample_data", glob="*.txt", show_progress=True)

In [32]:
parser = MyParser()
for blob in blob_loader.yield_blobs():  # Blob 로드
    for doc in parser.lazy_parse(blob):       # Blob 파싱
        print(doc)
        break

100%|██████████| 1/1 [00:00<00:00, 938.95it/s]

page_content='meow meow🐱 
' metadata={'line_number': 1, 'source': 'sample_data\\meow.txt'}


## Generic Loader
LangChain은 `GenericLoader` 추상화를 통해 `BaseBlobParser`와 `BlobLoader`를 함께 구성한다.

`GenericLoader`는 기존 `BlobLoader` 구현을 쉽게 사용할 수 있는 표준화된 클래스 방법을 제공한다. 현재는 `FileSystemBlobLoader`만 지원됩니다.


In [33]:
from langchain_community.document_loaders.generic import GenericLoader

loader = GenericLoader.from_filesystem(
    path="./sample_data",
    glob="*.txt",
    show_progress=True,
    parser=MyParser()
)

for idx, doc in enumerate(loader.lazy_load()):
    if idx < 5:
        print(doc)

print("... output truncated for demo purposes")

100%|██████████| 1/1 [00:00<00:00, 651.19it/s]

page_content='meow meow🐱 
' metadata={'line_number': 1, 'source': 'sample_data\\meow.txt'}
page_content=' meow meow🐱 
' metadata={'line_number': 2, 'source': 'sample_data\\meow.txt'}
page_content=' meow😻😻' metadata={'line_number': 3, 'source': 'sample_data\\meow.txt'}
... output truncated for demo purposes


## 커스텀 Generic Loader

클래스를 만드는 것을 정말 좋아한다면 하위 클래스를 만들어서 로직을 캡슐화하면 된다. 이 클래스를 하위 클래스로 사용하면 기존 로더를 사용하여 데이터를 로드할 수 있다.

In [34]:
from typing import Any

class MyCustomLoader(GenericLoader):
    @staticmethod
    def get_parser(**kwargs: Any) -> BaseBlobParser:
        """Override this method to associate a default parser with the class."""
        return MyParser()

In [35]:
loader = MyCustomLoader.from_filesystem(
    path="./sample_data/", 
    glob="*.txt", 
    show_progress=True
)

for idx, doc in enumerate(loader.lazy_load()):
    if idx < 5:
        print(doc)

print("... output truncated for demo purposes")

100%|██████████| 1/1 [00:00<00:00, 1000.31it/s]

page_content='meow meow🐱 
' metadata={'line_number': 1, 'source': 'sample_data\\meow.txt'}
page_content=' meow meow🐱 
' metadata={'line_number': 2, 'source': 'sample_data\\meow.txt'}
page_content=' meow😻😻' metadata={'line_number': 3, 'source': 'sample_data\\meow.txt'}
... output truncated for demo purposes
